<a href="https://colab.research.google.com/github/Byeongali/CS50/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: K-Nearest Neighbours


## Version history

| Date | Author | Description |
|:----:|:------:|:------------|
2021-01-18 | Josiah Wang | First version | 
2021-10-18 | Josiah Wang | Fixed typo in Distance Weighted K-NN Classifier evaluation. The last piece of code should say WeightedKNNClassifier(k) and not KNNClassifier(k), obviously! |

## Introduction

The aim of this lab exercise is to give you some practical experience to build a K-Nearest Neighbours (K-NN) classifier.

By the end of this lab exercise, you will have constructed different variants of a K-NN classifier as discussed in the lectures.

## The Iris dataset

Continuing where we left off in Lab 1, we will again work with the Iris dataset.

I will just copy and rerun my solutions from Lab 1. Feel free to copy your own implementations over.

In [ ]:
import os
import numpy as np
from numpy.random import default_rng

# Download iris data if it does not exist
if not os.path.exists("iris.data"):
    !wget -O iris.data https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data


def read_dataset(filepath):
    """ Read in the dataset from the specified filepath

    Args:
        filepath (str): The filepath to the dataset file

    Returns:
        tuple: returns a tuple of (x, y, classes), each being a numpy array. 
               - x is a numpy array with shape (N, K), 
                   where N is the number of instances
                   K is the number of features/attributes
               - y is a numpy array with shape (N, ), and should be integers from 0 to C-1
                   where C is the number of classes 
               - classes : a numpy array with shape (C, ), which contains the 
                   unique class labels corresponding to the integers in y
    """

    x = []
    y_labels = []
    for line in open(filepath):
        if line.strip() != "": # handle empty rows in file
            row = line.strip().split(",")
            x.append(list(map(float, row[:-1]))) 
            y_labels.append(row[-1])
    
    [classes, y] = np.unique(y_labels, return_inverse=True) 

    x = np.array(x)
    y = np.array(y)
    return (x, y, classes)


def split_dataset(x, y, test_proportion, random_generator=default_rng()):
    """ Split dataset into training and test sets, according to the given 
        test set proportion.
    
    Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)
        y (np.ndarray): Class labels, numpy array with shape (N,)
        test_proprotion (float): the desired proportion of test examples 
                                 (0.0-1.0)
        random_generator (np.random.Generator): A random generator

    Returns:
        tuple: returns a tuple of (x_train, x_test, y_train, y_test) 
               - x_train (np.ndarray): Training instances shape (N_train, K)
               - x_test (np.ndarray): Test instances shape (N_test, K)
               - y_train (np.ndarray): Training labels, shape (N_train, )
               - y_test (np.ndarray): Test labels, shape (N_train, )
    """

    shuffled_indices = random_generator.permutation(len(x))
    n_test = round(len(x) * test_proportion)
    n_train = len(x) - n_test
    x_train = x[shuffled_indices[:n_train]]
    y_train = y[shuffled_indices[:n_train]]
    x_test = x[shuffled_indices[n_train:]]
    y_test = y[shuffled_indices[n_train:]]
    return (x_train, x_test, y_train, y_test)



(x, y, classes) = read_dataset("iris.data")

seed = 60012
rg = default_rng(seed)
x_train, x_test, y_train, y_test = split_dataset(x, y, 
                                                 test_proportion=0.2, 
                                                 random_generator=rg)
print(x_train.shape)
print(x_test.shape)


## K-NN Classifier

In Lab 1, you have constructed a (one) Nearest Neighbour classifier.

We will now try to generalise the nearest neighbour classifier as a **K-Nearest Neighours (K-NN)** classifier. For each test example, the classifier will predict the majority class label among the $K$ nearest training examples, again according to the Euclidean distance metric $d(x^{(i)}, x^{(q)})=\sqrt{\sum_f^F (x_f^{(i)} - x_f^{(q)})^2}$. If there is draw, choose one of the majority class labels arbitrarily, or at random.

Complete the `predict()` method of the `KNNClassifier` class. Note that the class now takes an optional hyperparameter `k` in its constructor.


In [ ]:
class KNNClassifier:
    def __init__(self, k=5):
        """ K-NN Classifier.

        Args:
        k (int): Number of nearest neighbours. Defaults to 5.
        """
        self.k = k
        self.x = np.array([])
        self.y = np.array([])

    def fit(self, x, y):
        """ Fit the training data to the classifier.

        Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)
        y (np.ndarray): Class labels, numpy array with shape (N,)
        """
        self.x = x
        self.y = y

    def predict(self, x):
        """ Perform prediction given some examples.

        Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)

        Returns:
        y (np.ndarray): Predicted class labels, numpy array with shape (N,)
        """ 

        # TODO: Implement a K-NN classifier



In [ ]:
nn_classifier = KNNClassifier(k=5)
nn_classifier.fit(x_train, y_train)
nn_predictions = nn_classifier.predict(x_test)
print(nn_predictions)

### Evaluation

Now, let's evaluate the accuracy of our K-NN classifier. We will test K from 1 to 20. Again, you should be able to achieve very high accuracy.

In [ ]:
def compute_accuracy(y_gold, y_prediction):
    """ Compute the accuracy given the ground truth and predictions

    Args:
    y_gold (np.ndarray): the correct ground truth/gold standard labels
    y_prediction (np.ndarray): the predicted labels

    Returns:
    float : the accuracy
    """

    assert len(y_gold) == len(y_prediction)  
    
    try:
        return np.sum(y_gold == y_prediction) / len(y_gold)
    except ZeroDivisionError:
        return 0

In [ ]:
for k in range(1, 21):
    nn_classifier = KNNClassifier(k)
    nn_classifier.fit(x_train, y_train)
    nn_predictions = nn_classifier.predict(x_test)
    accuracy = compute_accuracy(y_test, nn_predictions)
    print(f"K={k}: {accuracy}")

## Distance Weighted K-NN Classifier

Now, let us implement the **distance weighted** K-NN classifier as discussed in the lectures. 
Pleae complete the `predict()` method for the `WeightedKNNClassifier` class below.

For each test example $x^{(q)}$, you will need to compute the weight $w^{(i)}_{q}$ for each training example $x^{(i)}$. 

Then for each class label, sum the weights of the $K$ nearest examples. Assign the test example to the class label with largest sum.

The weight can be anything reasonable. For this tutorial, we will use the weight $w_{q}^{(i)}=\frac{1}{d(x^{(i)}, x^{(q)})}$, where $d(x^{(i)}, x^{(q)})=\sqrt{\sum_f^F (x_f^{(i)} - x_f^{(q)})^2}$ is the Euclidean distance. 


In [ ]:
class WeightedKNNClassifier:
    def __init__(self, k=5):
        """ K-NN Classifier.

        Args:
        k (int): Number of nearest neighbours. Defaults to 5.
        """
        self.k = k
        self.x = np.array([])
        self.y = np.array([])

    def fit(self, x, y):
        """ Fit the training data to the classifier.

        Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)
        y (np.ndarray): Class labels, numpy array with shape (N,)
        """
        self.x = x
        self.y = y

    def predict(self, x):
        """ Perform prediction given some examples.

        Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)

        Returns:
        y (np.ndarray): Predicted class labels, numpy array with shape (N,)
        """ 

        # TODO: Implement a distance weighted K-NN classifier



In [ ]:
nn_classifier = WeightedKNNClassifier(k=5)
nn_classifier.fit(x_train, y_train)
nn_predictions = nn_classifier.predict(x_test)
print(nn_predictions)

### Evaluation

Again, we will evaluate the accuracy of the distance weighted K-NN classifier for K=1 to 20. As this is a small and relatively easy dataset, you should get similar results as with the simple K-NN classifier.

You might observe the benefits of the distance weighted K-NN classifier better when training on larger and noisier datasets.

In [ ]:
for k in range(1, 21):
    nn_classifier = WeightedKNNClassifier(k)
    nn_classifier.fit(x_train, y_train)
    nn_predictions = nn_classifier.predict(x_test)
    accuracy = compute_accuracy(y_test, nn_predictions)
    print(f"K={k}: {accuracy}")

## Summary

You have built a K-Nearest Neighbour classifier (and the distance-weighted variant)! Because the Iris dataset is a small and simple dataset, you may only observe a small improvement in accuracy (if any) compared to a simple one-nearest neighbour classifier.

And that is it for this week's lab tutorials. We will not have a decision tree tutorial for this course. The coursework itself will contain a short guide to help you think about implementing your own decision trees.

In the next lab tutorial, you will be implementing different evaluation metrics and performing cross-validation.
